In [ ]:
import pandas as pd  
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
import warnings
warnings.simplefilter("ignore")
import re
import gensim
from gensim.models import Word2Vec
import numpy as np
#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

 

data_frame = pd.read_excel("(Jackie Global) Global Services Comments.xlsx")
data_frame = data_frame.rename(columns={'If there is one thing that would improve your [Performance, Speed of Decision-making, Innovation ] what would it be (English)?': 'Comment'})
data_frame.head()

 

Innovation_df = data_frame[data_frame["Question"] == "Innovation"]
Innovation_df.head()

 

#convert to lowercase, strip and remove punctuations
def preprocess(text):
    text = text.lower() 
    text=text.strip()  
    text=re.compile('<.*?>').sub('', text) 
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)  
    text = re.sub('\s+', ' ', text)  
    text = re.sub(r'\[[0-9]*\]',' ',text) 
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text) 
    text = re.sub(r'\s+',' ',text) 
    return text

 

 
# STOPWORD REMOVAL
def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)
#LEMMATIZATION
# Initialize the lemmatizer
wl = WordNetLemmatizer()
 
# This is a helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
# Tokenize the sentence
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)

 

def finalpreprocess(string):
    return lemmatizer(stopword(preprocess(string)))
Innovation_df['Comment_clean'] = Innovation_df['Comment'].apply(lambda x: finalpreprocess(x))

 


Innovation_df_clean=Innovation_df.iloc[:,3:]
Innovation_df_clean.head()

 

# Word2Vec runs on tokenized sentences
Innovation_df_token= [nltk.word_tokenize(i) for i in Innovation_df_clean["Comment_clean"]]

 

#building Word2Vec model
class MeanEmbeddingVectorizer:
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(next(iter(word2vec.values())))
    def fit(self, X, y):
        return self
    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

 

Innovation_df_clean['Innovation_df_tok']=[nltk.word_tokenize(i) for i in Innovation_df_clean["Comment_clean"]] 
model = Word2Vec(Innovation_df_clean['Innovation_df_tok'],min_count=1)
w2v = dict(zip(model.wv.index_to_key , model.wv.vectors))
modelw = MeanEmbeddingVectorizer(w2v)
# converting text to numerical data using Word2Vec
Innovation_df_token_w2v = modelw.transform(Innovation_df_token)

 

# Kmean_model_word2vec = KMeans(n_clusters=10, init='k-means++', max_iter=200, n_init=15,random_state=20)
# Kmean_model_word2vec.fit(performance_df_token_w2v)

 

import pickle

 

with open("Kmeans_final_word2vec_model.pickle", "rb") as f:
    Kmean_model_word2vec = pickle.load(f)
    
Kmean_word2vec_clus= Kmean_model_word2vec.fit_predict(Innovation_df_token_w2v)

 

Innovation_df_clean["cluster"]= Kmean_word2vec_clus.tolist()

 

Innovation_df_clean["cluster"].value_counts()

 


Innovation_df_clean[["Comment_clean","cluster"]].to_csv("ae.csv")

 

## Agglomerative clustering

 

import pickle

 

with open("Agg_final_word2vec_model.pkl", "rb") as f:
    Agg_model_word2vec = pickle.load(f)
    
Kmean_word2vec_clus= Agg_model_word2vec.fit_predict(Innovation_df_token_w2v)

 

Innovation_df_clean["cluster_agg"]= Kmean_word2vec_clus.tolist()

 

Innovation_df_clean["cluster_agg"].value_counts()

 

Decision_df_clean[["Comment_clean","cluster_agg"]].to_csv("af.csv")